In [1]:
%pip install requests beautifulsoup4 python-dotenv

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement tkinter (from versions: none)
ERROR: No matching distribution found for tkinter


In [3]:
import requests
from bs4 import BeautifulSoup
import re
import logging
import tkinter as tk
from tkinter import scrolledtext
import os
from dotenv import load_dotenv
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
import torch

# Load environment variables from .env file
load_dotenv()

# Get API key and custom search engine ID from environment variables
API_KEY = os.getenv('GOOGLE_API_KEY')
CX = os.getenv('GOOGLE_CX')

# Set up logging
logging.basicConfig(level=logging.INFO)

# Load pre-trained DistilBERT model and tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')

def google_search(query: str):
    """Perform a Google search using the Custom Search JSON API."""
    url = f'https://www.googleapis.com/customsearch/v1?q={query}&key={API_KEY}&cx={CX}'
    try:
        response = requests.get(url)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        logging.error(f"Request failed: {e}")
        return {}

def get_top_results(results):
    """Extract top search results from the Google search API response."""
    items = results.get('items', [])
    return [{'title': item.get('title'), 'link': item.get('link'), 'snippet': item.get('snippet')}
            for item in items[:5]]

def clean_text(text):
    """Clean and normalize the text."""
    return re.sub(r'\s+', ' ', text).strip()

def scrape_website(url):
    """Scrape text content from a given website URL."""
    try:
        response = requests.get(url, timeout=10)
        soup = BeautifulSoup(response.text, 'html.parser')
        for script in soup(["script", "style"]):
            script.decompose()
        text = soup.get_text()
        lines = (line.strip() for line in text.splitlines())
        chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
        text = '\n'.join(chunk for chunk in chunks if chunk)
        logging.info(f"Successfully scraped content from {url}")
        return clean_text(text)
    except requests.exceptions.RequestException as e:
        logging.error(f"Failed to retrieve {url}: {e}")
        return ""

def analyze_text(text: str) -> torch.Tensor:
    """Analyze text using DistilBERT."""
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.logits

# GUI Functions
def perform_search():
    query = entry.get()
    results = google_search(query)
    top_results = get_top_results(results)
    listbox.delete(0, tk.END)
    for result in top_results:
        listbox.insert(tk.END, result['title'])

def show_content(event):
    selected = listbox.curselection()
    if selected:
        index = selected[0]
        url = get_top_results(google_search(entry.get()))[index]['link']
        content = scrape_website(url)
        text_area.delete('1.0', tk.END)
        text_area.insert(tk.END, content)

        # Analyze content with DistilBERT
        analysis = analyze_text(content)
        logging.info(f"Text analysis result: {analysis}")

# Create the GUI
root = tk.Tk()
root.title("Simple Browser")

# Search Bar
tk.Label(root, text="Search:").pack(padx=10, pady=5)
entry = tk.Entry(root, width=50)
entry.pack(padx=10, pady=5)
tk.Button(root, text="Search", command=perform_search).pack(pady=5)

# Results Listbox
listbox = tk.Listbox(root, width=100, height=10)
listbox.pack(padx=10, pady=5)
listbox.bind("<Double-1>", show_content)

# Content Display
text_area = scrolledtext.ScrolledText(root, width=100, height=20)
text_area.pack(padx=10, pady=5)

root.mainloop()


x:\Ly.JSxPY\deps\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
x:\Ly.JSxPY\deps\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
INFO:root:Successfully